In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2
from collections import deque
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
train_copy = train.copy()

In [3]:
def Correct_Square(train_copy):
    
    
    
    null_list = train_copy.loc[train_copy.LifeSquare.isnull()].index   
   
    for i in null_list:
        
        train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square'] - (train_copy.loc[i]['Square'] / 100 * 15) - \
                                        train_copy.loc[i]['KitchenSquare']
                                            
    ID_List = train_copy.loc[train_copy['LifeSquare'] > train_copy['Square']].index
    for i in ID_List:
        temp = train_copy.loc[i]['LifeSquare']
        train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square']
        train_copy.loc[i, 'Square'] = temp
    
    
    indexoid = train_copy.loc[train_copy.Square - train_copy.LifeSquare > train_copy.LifeSquare * 2].index
    
    for i in indexoid:
        
        train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square'] - (train_copy.loc[i]['Square'] / 100 * 20) 
            
            
            

In [4]:
def Correct_floors(train_copy):
    ID_List = []
    ID_List.extend(train_copy.loc[train_copy['Floor'] > train_copy['HouseFloor']].index)
    for i in ID_List:
        temp = train_copy.loc[i]['Floor']
        train_copy.loc[i, 'Floor'] = train_copy.loc[i]['HouseFloor']
        train_copy.loc[i, 'HouseFloor'] = temp


In [5]:
def Correct_other(train_copy):
    train_copy.loc[train_copy['HouseYear'] > 2019, 'HouseYear'] = train_copy['HouseYear'].median()    
    train_copy.loc[train_copy.KitchenSquare > train_copy.Square, 'KitchenSquare'] = train_copy.loc[train_copy.KitchenSquare > train_copy.Square]['Square'] - train_copy.loc[train_copy.KitchenSquare > train_copy.Square]['LifeSquare']
    train_copy.loc[train_copy.HouseFloor > 50, 'HouseFloor'] = train_copy.loc[train_copy.HouseFloor > 50]['Floor'].median() + 10
    train_copy.loc[(train_copy.Rooms > 5) & (train_copy.loc[train_copy.Rooms > 5]['Square'] < train_copy.loc[train_copy.Rooms == 5]['Square'].mean()), 'Rooms'] = train_copy.loc[train_copy.Rooms]['Rooms'].median() 
    train_copy.loc[train_copy.Square > (train_copy.loc[train_copy.Rooms, 'Square'].mean() + 100), 'Square'] = train_copy.loc[train_copy.Rooms]['Square'].mean()
    train_copy.drop('Healthcare_1', axis=1, inplace=True)

In [6]:
features = ['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2',
       'Shops_1']
target = ['Price']

In [7]:

def prepare_features(train_copy):
     
    Correct_floors(train_copy)
    Correct_other(train_copy)
    Correct_Square(train_copy)
    

In [8]:
prepare_features(train_copy)

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(train_copy[features], train_copy[target], \
                                                      test_size=0.3, random_state=42)

In [11]:
model = RFR(n_estimators=5, max_depth=12, random_state=42)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred = model.predict(X_valid)
print('Валидационный',r2(y_valid, y_pred),'тренировочный', r2(y_train, y_pred_train))

Валидационный 0.6526661414871289 тренировочный 0.8767731102769598


In [12]:
def Testing_models():
    
    r2_value = 0
    estimator = 0
    depth = 0
    
    for n_estimators in range(1, 16):
        for max_depth in range(1, 25, 2):
            model = RFR(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            if r2_value < r2(y_valid, y_pred):
                r2_value = r2(y_valid, y_pred)
                estimator = n_estimators
                depth = max_depth
           
    
    return estimator, depth, r2_value

In [13]:
def Testing_models_extended(X_train, X_valid, y_train, y_valid):
    
    r2_value = 0
    estimator = 0
    depth = 0
    
    for n_estimators in range(1, 16):
        for max_depth in range(1, 25, 2):
            model = RFR(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            if r2_value < r2(y_valid, y_pred):
                r2_value = r2(y_valid, y_pred)
                estimator = n_estimators
                depth = max_depth
    
    
    
    return estimator, depth, r2_value

In [14]:
n_estimator, max_depth, r2_value = Testing_models()
n_estimator, max_depth, r2_value

(14, 15, 0.6884433979092972)

In [15]:
best_r2_yet = r2_value

In [16]:
final_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [17]:

def Testing_features(features, best_r2_yet):
    
    test_features = deque()

    flag = True
    r2_best = 0
    test_features = deque(features.copy())
    del_list = []
    must_del_feat = None
    best_r2_yet = best_r2_yet
    
    while flag:
        
        for i in range(len(test_features)):
            
            del_list.append(test_features.pop())
            
            final_model.fit(X_train[test_features], y_train)
            y_pred = final_model.predict(X_valid[test_features])
            
            if r2_best < r2(y_valid, y_pred):
                
                r2_best = r2(y_valid, y_pred)
                
                must_del_feat = del_list[0]
                
            test_features.appendleft(del_list.pop())
        
        if best_r2_yet < r2_best:
            
            best_r2_yet = r2_best
            test_features.remove(must_del_feat)
            
        else:
            flag = False
                
    
    
    return test_features, best_r2_yet

In [18]:

Testing_features(features, best_r2_yet)


(deque(['Id',
        'DistrictId',
        'Square',
        'LifeSquare',
        'KitchenSquare',
        'Floor',
        'HouseFloor',
        'HouseYear',
        'Ecology_1',
        'Social_1',
        'Social_2',
        'Social_3',
        'Helthcare_2',
        'Shops_1']),
 0.6962415265935711)

In [19]:
all_feattures = train_copy[train_copy.columns[:-1]].copy()

In [20]:
all_feattures = pd.get_dummies(all_feattures)

In [21]:
all_X_train, all_X_valid, all_y_train, all_y_valid = train_test_split(all_feattures, train_copy['Price'], \
                                                      test_size=0.3, random_state=42)

all_feat_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [22]:
all_feat_model.fit(all_X_train, all_y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=14,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [23]:
all_y_predict = all_feat_model.predict(all_X_valid)
all_r2_best_yet = r2(all_y_valid, all_y_predict)
all_r2_best_yet

0.6840998774097546

In [24]:
def Test_all_feats(features, best_r2_yet):
    flag = True
    r2_best = best_r2_yet
    test_features = deque(features.copy())
    del_list = []
    must_del_feat = None
    final_list = []
    best_model = None
    
    
    while flag:
        
        for i in range(len(test_features)):
            
            
            del_list.append(test_features.pop())
            
            all_X_train, all_X_valid, all_y_train, all_y_valid = train_test_split(all_feattures[test_features], train_copy[target], \
                                                      test_size=0.3, random_state=42)

            all_feat_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)
    
            
            
            all_feat_model.fit(all_X_train, all_y_train)
            
            y_pred = all_feat_model.predict(all_X_valid)
    
    
    
            
            if r2_best < r2(all_y_valid, y_pred):
                
                r2_best = r2(all_y_valid, y_pred)
                
                must_del_feat = del_list[0]
                
                best_model = all_feat_model
                
                print(r2(all_y_valid, y_pred))
                print(len(test_features))
                print(test_features)
                
                final_list = test_features.copy()
                print()
                
            test_features.appendleft(del_list.pop())
        
        if best_r2_yet < r2_best:
            
            best_r2_yet = r2_best
            test_features.remove(must_del_feat)
                
            
        else:
            flag = False
                

    
    return final_list, best_r2_yet, len(test_features), best_model

In [25]:
optimal_features, R2_score_, feat_list_len, best_model = Test_all_feats(all_feattures, all_r2_best_yet)
optimal_features

0.6872452667439818
20
deque(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A'])

0.6881746050643033
20
deque(['Ecology_3_B', 'Shops_2_A', 'Shops_2_B', 'Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B'])

0.6908803649692263
20
deque(['Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B', 'Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1'])

0.6919369263324949
20
deque(['Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_

deque(['Ecology_3_B',
       'Shops_2_A',
       'Shops_2_B',
       'DistrictId',
       'Rooms',
       'Square',
       'HouseFloor',
       'HouseYear',
       'Ecology_1',
       'Social_3',
       'Helthcare_2',
       'Shops_1',
       'Ecology_2_A',
       'Ecology_2_B'])

In [31]:
R2_score_, feat_list_len


(0.7194724360518145, 14)

#### По сути здесь идет работа по оптимизации, какие признаки оставить для предсказания, сколько деревьев надо применить в RFR для оптимального предсказания итп. Также идет работа по выбросам.